<a href="https://colab.research.google.com/github/RubenGres/AerialDreams/blob/main/train_controlnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


In [1]:
from accelerate.utils import write_basic_config
write_basic_config()

Configuration already exists at /home/innani/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


/home/innani/miniconda3/envs/Seg2Sat-Training/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [2]:
# ==============================================================================
#           CÉLULA DE CONFIGURAÇÃO FINAL
# ==============================================================================
print("Carregando configurações para o fine-tuning local...")

# 1. Definição dos Modelos Base
SD_MODEL = "stabilityai/stable-diffusion-2-1-base"
CONTROLNET_MODEL = "rgres/Seg2Sat-sd-controlnet"

# 2. Definição do Dataset e Saída
# ATUALIZADO com o nome da sua pasta: 'pinus_dataset'
DATASET_DIR = "../../pinus_dataset"  # O caminho relativo de 'finetuning/controlnet_training/' para a raiz e depois para 'pinus_dataset'.
MODEL_OUT = "model_pinus_finetuned"

# 3. Nomes das Colunas do Dataset
IMG_COL = "image"
CONDITIONING_IMG_COL = "conditioning_image"
CAPTION_COL = "prompt"

# 4. Parâmetros de Treinamento
LEARNING_RATE = 1e-5
BATCH_SIZE = 1
EPOCHS = 150
CKPT_STEPS = 200

# 5. Nome do Projeto para Monitoramento
PROJECT_NAME = "seg2sat-pinus-finetune"

print("✅ Configuração concluída e variáveis carregadas.")

Carregando configurações para o fine-tuning local...
✅ Configuração concluída e variáveis carregadas.


In [3]:
# ==============================================================================
#           CÉLULA DE PREPARAÇÃO E EXECUÇÃO FINAL (VERSÃO FINALÍSSIMA)
# ==============================================================================
import pandas as pd
import json
import os

print("Preparando o arquivo metadata.jsonl a partir do seu metadata.csv...")
# Carrega o seu CSV
df = pd.read_csv(f'{DATASET_DIR}/metadata.csv')

# Cria o arquivo metadata.jsonl, que é o formato ideal para o script de treino
with open(f'{DATASET_DIR}/metadata.jsonl', 'w') as f:
    for index, row in df.iterrows():
        # Gera o nome do arquivo da máscara a partir do nome do arquivo da imagem
        mask_filename = os.path.splitext(row['filename'])[0] + '.png'
        
        # Cria o registro no formato JSON com os nomes de coluna corretos
        record = {
            "file_name": f"images/{row['filename']}", # <-- ADICIONADO: A chave 'file_name' que estava faltando
            IMG_COL: f"images/{row['filename']}",
            CONDITIONING_IMG_COL: f"masks/{mask_filename}",
            CAPTION_COL: row['prompt']
        }
        f.write(json.dumps(record) + '\n')
        
print(f"✅ Arquivo metadata.jsonl recriado com a chave 'file_name' em '{DATASET_DIR}/'")

Preparando o arquivo metadata.jsonl a partir do seu metadata.csv...
✅ Arquivo metadata.jsonl recriado com a chave 'file_name' em '../../pinus_dataset/'


In [6]:
# Constrói o comando final para o accelerate
cmd = f"""accelerate launch train_controlnet.py \
 --pretrained_model_name_or_path="{SD_MODEL}" \
 --controlnet_model_name_or_path="{CONTROLNET_MODEL}" \
 --train_data_dir="{DATASET_DIR}" \
 --output_dir="{MODEL_OUT}" \
 --conditioning_image_column="{CONDITIONING_IMG_COL}" \
 --image_column="{IMG_COL}" \
 --caption_column="{CAPTION_COL}" \
 --resolution=512 \
 --learning_rate={LEARNING_RATE} \
 --train_batch_size={BATCH_SIZE} \
 --num_train_epochs={EPOCHS} \
 --tracker_project_name="{PROJECT_NAME}" \
 --enable_xformers_memory_efficient_attention \
 --checkpointing_steps={CKPT_STEPS} \
 --report_to="tensorboard"
"""

print("-----------------------------------------")
print("✅ Comando de treinamento pronto para ser executado:")
print(cmd)
print("-----------------------------------------")


-----------------------------------------
✅ Comando de treinamento pronto para ser executado:
accelerate launch train_controlnet.py  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1-base"  --controlnet_model_name_or_path="rgres/Seg2Sat-sd-controlnet"  --train_data_dir="../../pinus_dataset"  --output_dir="model_pinus_finetuned"  --conditioning_image_column="conditioning_image"  --image_column="image"  --caption_column="prompt"  --resolution=512  --learning_rate=1e-05  --train_batch_size=1  --num_train_epochs=150  --tracker_project_name="seg2sat-pinus-finetune"  --enable_xformers_memory_efficient_attention  --checkpointing_steps=200  --report_to="tensorboard"

-----------------------------------------


In [4]:
# ==============================================================================
#           VERSÃO DO COMANDO PARA CONTINUAR O TREINAMENTO
# ==============================================================================

# Defina o nome do checkpoint a partir do qual você quer continuar
# Pode ser um número específico ou "latest" para o mais recente
checkpoint_para_continuar = "latest" 

cmd = f"""accelerate launch train_controlnet.py \
 --pretrained_model_name_or_path="{SD_MODEL}" \
 --controlnet_model_name_or_path="{CONTROLNET_MODEL}" \
 --train_data_dir="{DATASET_DIR}" \
 --output_dir="{MODEL_OUT}" \
 --conditioning_image_column="{CONDITIONING_IMG_COL}" \
 --image_column="{IMG_COL}" \
 --caption_column="{CAPTION_COL}" \
 --resolution=512 \
 --learning_rate={LEARNING_RATE} \
 --train_batch_size={BATCH_SIZE} \
 --num_train_epochs={EPOCHS} \
 --tracker_project_name="{PROJECT_NAME}" \
 --enable_xformers_memory_efficient_attention \
 --checkpointing_steps={CKPT_STEPS} \
 --report_to="tensorboard" \
 --resume_from_checkpoint="{checkpoint_para_continuar}"
"""
# A linha acima foi adicionada. O valor "latest" instrui o accelerate a encontrar
# automaticamente o último checkpoint salvo na pasta de saída.

print("-----------------------------------------")
print(f"✅ Comando para CONTINUAR o treinamento a partir do checkpoint '{checkpoint_para_continuar}':")
print(cmd)
print("-----------------------------------------")

-----------------------------------------
✅ Comando para CONTINUAR o treinamento a partir do checkpoint 'latest':
accelerate launch train_controlnet.py  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1-base"  --controlnet_model_name_or_path="rgres/Seg2Sat-sd-controlnet"  --train_data_dir="../../pinus_dataset"  --output_dir="model_pinus_finetuned"  --conditioning_image_column="conditioning_image"  --image_column="image"  --caption_column="prompt"  --resolution=512  --learning_rate=1e-05  --train_batch_size=1  --num_train_epochs=150  --tracker_project_name="seg2sat-pinus-finetune"  --enable_xformers_memory_efficient_attention  --checkpointing_steps=200  --report_to="tensorboard"  --resume_from_checkpoint="latest"

-----------------------------------------


In [7]:
# Execute esta célula para iniciar o fine-tuning!
!{cmd}

ipex flag is deprecated, will be removed in Accelerate v1.10. From 2.7.0, PyTorch has all needed optimizations for Intel CPU and XPU.
07/04/2025 13:50:36 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'dynamic_thresholding_ratio', 'sample_max_value', 'rescale_betas_zero_snr', 'thresholding', 'variance_type', 'clip_sample_range', 'timestep_spacing'} was not found in config. Values will be initialized to default values.
{'latents_std', 'mid_block_add_attention', 'use_quant_conv', 'latents_mean', 'shift_factor', 'force_upcast', 'use_post_quant_conv', 'scaling_factor'} was not found in config. Values will be initialized to default values.
All model checkpoint weights were used when initializing AutoencoderKL.

All the weights of Autoenco